In [1]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense

In [2]:
# Carga en memoria el dataframe
df = pd.read_pickle('affective_dataset_v2.pkl') 
#df = pd.read_pickle('3Emociones.pkl') 

In [3]:
df.head()
#print(df.columns)

,image,face_closeup,face_landmarks,label
0,"[[165, 166, 164, 167, 167, 166, 167, 167, 167,...","[[161, 160, 161, 161, 161, 161, 161, 160, 99, ...","[[62, 83], [62, 88], [63, 93], [63, 97], [64, ...",bored
1,"[[165, 166, 165, 166, 167, 167, 167, 167, 167,...","[[160, 161, 160, 161, 161, 162, 161, 162, 132,...","[[62, 83], [62, 88], [63, 93], [63, 97], [64, ...",bored
2,"[[165, 165, 165, 166, 167, 166, 166, 167, 167,...","[[160, 161, 161, 60, 36, 76, 88, 88, 92, 102, ...","[[63, 82], [63, 87], [63, 91], [63, 96], [65, ...",bored
3,"[[165, 166, 165, 165, 166, 167, 167, 167, 167,...","[[160, 161, 160, 45, 42, 81, 88, 88, 92, 101, ...","[[63, 83], [63, 87], [63, 92], [64, 97], [65, ...",bored
4,"[[165, 166, 166, 166, 166, 167, 166, 166, 167,...","[[160, 160, 160, 41, 43, 81, 88, 88, 92, 102, ...","[[63, 83], [63, 88], [63, 92], [64, 97], [65, ...",bored


In [4]:
# Eliminar filas donde la columna 'label' tiene valores 'focused', 'interested', 'relaxed'
df = df[~df['label'].isin(['focused', 'interested', 'relaxed'])]

# Verificar el resultado
df.head()


,image,face_closeup,face_landmarks,label
0,"[[165, 166, 164, 167, 167, 166, 167, 167, 167,...","[[161, 160, 161, 161, 161, 161, 161, 160, 99, ...","[[62, 83], [62, 88], [63, 93], [63, 97], [64, ...",bored
1,"[[165, 166, 165, 166, 167, 167, 167, 167, 167,...","[[160, 161, 160, 161, 161, 162, 161, 162, 132,...","[[62, 83], [62, 88], [63, 93], [63, 97], [64, ...",bored
2,"[[165, 165, 165, 166, 167, 166, 166, 167, 167,...","[[160, 161, 161, 60, 36, 76, 88, 88, 92, 102, ...","[[63, 82], [63, 87], [63, 91], [63, 96], [65, ...",bored
3,"[[165, 166, 165, 165, 166, 167, 167, 167, 167,...","[[160, 161, 160, 45, 42, 81, 88, 88, 92, 101, ...","[[63, 83], [63, 87], [63, 92], [64, 97], [65, ...",bored
4,"[[165, 166, 166, 166, 166, 167, 166, 166, 167,...","[[160, 160, 160, 41, 43, 81, 88, 88, 92, 102, ...","[[63, 83], [63, 88], [63, 92], [64, 97], [65, ...",bored


In [5]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

In [6]:
x = np.array(df['face_landmarks'].tolist())

In [7]:
print(x.shape)

(4623, 144, 2)


In [8]:
print("Original labels:", df['label'].unique())
print("Encoded labels:", label_encoder.classes_)

Original labels: ['bored' 'engaged' 'excited']
Encoded labels: ['bored' 'engaged' 'excited']


In [9]:
labels = df['label'].astype('category').cat.codes
print(labels)

0       0
1       0
2       0
3       0
4       0
       ..
4618    2
4619    2
4620    2
4621    2
4622    2
Length: 4623, dtype: int8


In [10]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)


#X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,stratify=y, random_state=42)

In [11]:
from keras.layers import Input, Flatten, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

# Entrada del modelo
inputs = Input(shape=(144, 2))

# Aplanar los datos de entrada
flatten = Flatten()(inputs)

# Capas ocultas
dense1 = Dense(600, activation='relu')(flatten)
dense2 = Dense(480, activation='relu')(dense1)
dense3 = Dense(64, activation='relu')(dense2)


# Capa de salida
outputs = Dense(len(label_encoder.classes_), activation='softmax')(dense3)

model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
import tensorflow as tf
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') >= 0.6:
            print("\nReached 60% accuracy so cancelling training!")
            self.model.stop_training = True
            
back = myCallback() 

In [13]:
model.fit(X_train, y_train, epochs=800, batch_size=32, validation_split=0.2, callbacks = [back])

Epoch 1/800
81/81 [==============================] - 2s 7ms/step - loss: 6.5360 - accuracy: 0.3717 - val_loss: 1.4907 - val_accuracy: 0.3410
Epoch 2/800
81/81 [==============================] - 0s 5ms/step - loss: 1.3036 - accuracy: 0.4049 - val_loss: 1.0986 - val_accuracy: 0.3904
Epoch 3/800
81/81 [==============================] - 0s 5ms/step - loss: 1.2492 - accuracy: 0.3937 - val_loss: 1.4713 - val_accuracy: 0.4136
Epoch 4/800
81/81 [==============================] - 0s 5ms/step - loss: 1.2961 - accuracy: 0.3876 - val_loss: 1.1667 - val_accuracy: 0.4151
Epoch 5/800
81/81 [==============================] - 0s 5ms/step - loss: 1.1121 - accuracy: 0.4173 - val_loss: 1.1581 - val_accuracy: 0.3904
Epoch 6/800
81/81 [==============================] - 0s 5ms/step - loss: 1.1026 - accuracy: 0.4096 - val_loss: 1.2458 - val_accuracy: 0.3704
Epoch 7/800
81/81 [==============================] - 0s 5ms/step - loss: 1.1163 - accuracy: 0.4158 - val_loss: 1.1134 - val_accuracy: 0.3611
Epoch 8/800
8

In [14]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

44/44 [==============================] - 0s 4ms/step - loss: 1.0631 - accuracy: 0.4420
Loss: 1.063097357749939, Accuracy: 0.4419610798358917


In [15]:
#model.save('affective_MLP_trainned_000acc_6emo_stratified.h5')
model.save('affective_MLP_trainned_000acc_3emo.h5')